# METABRIC Dataset

paper [here](https://www.nature.com/articles/ncomms11479)

dataset [here](https://www.kaggle.com/datasets/raghadalharbi/breast-cancer-gene-expression-profiles-metabric)


Project Description:

This notebook summarizes the key scientific inquiries used in the context of a pharmaceutical company for analyzing clinical trials. Its purpose is to offer a comprehensive look into the data analysis processes within a pharmaceutical company, with a specific emphasis on biomarker discovery. It also includes a comparison of the same analysis executed in both R and Python.

Analysis agenda:

This notebook will cover the following aspect:

1. Exploratory Data analysis [?]
2. Survival Analysis
3. Differential Expression Analysis
4. Mutation [?]

In [ ]:
# notebook setup
## impoort packages
import collections
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
%matplotlib inline

## Exploratiry Data Analysis

Here we first look at the samples and features to have an idea of what type of data we are working with.

In [ ]:
# load METABRIC dataset
## load data
metabric_data = pd.read_csv(
    '../data/raw/METABRIC_RNA_Mutation.csv', # path to data
    index_col=0, # use first column as index
    low_memory=False # avoid warning
    )
## show header
metabric_data.head()    

## separate dataset

The csv file is a all in one table with different data types. The code below breaks the dataset into column subsets.

In [ ]:
# from 0 to 30 are the clinical data
clinical = metabric_data.iloc[:, :30].add_suffix("_meta")
print(f'clinical data shape: {clinical.shape}')
# from 30 to 519 are the gene expression data
expression = metabric_data.iloc[:, 30:519].add_suffix("_expr")
print(f'expression data shape: {expression.shape}')
# from 519 to end are the mutation data
mutation = metabric_data.iloc[:, 519:]
print(f'mutation data shape: {mutation.shape}')

## Clinical Data Exploration

Perform an high level exploration of the clinical features, with main focus on survival.

In [ ]:
clinical.head(5)

In [ ]:
clinical.info()

In [ ]:
# recode variables in overall_survival_meta
# to categorical variables
mapping = { 0 : "dead", 1 : "alive" }
# apply the mapping to the column
clinical["overall_survival_meta"] = clinical["overall_survival_meta"].map(mapping)


In [ ]:
# Create the figure and specify the number of subplots and their arrangement (2 rows, 2 columns)
fig, ax = plt.subplots(3, 2, figsize=(15, 14))
# Plot the distribution of age at diagnosis in the first subplot
sns.histplot(
    data=clinical, 
    x="age_at_diagnosis_meta", 
    hue="overall_survival_meta",
    ax=ax[0, 0]
    )
# Plot the distribution of total mutations in the second subplot
sns.histplot(
    data=clinical, 
    x="mutation_count_meta",
    hue="overall_survival_meta",
    ax=ax[0, 1]
    )
# Plot the distribution of tumour size in the third subplot
sns.histplot(
    data=clinical, 
    x="tumor_size_meta", 
    hue="overall_survival_meta",
    ax=ax[1, 0]
)
# Plot the distribution of age at diagnosis in the fourth subplot
sns.boxplot(
    data=clinical, 
    x="overall_survival_meta", 
    y="age_at_diagnosis_meta",
    ax=ax[1, 1]
    )
# plot the correlation between age at diagnosis and total mutations
sns.scatterplot(
    data=clinical,
    x="tumor_size_meta",
    y="mutation_count_meta",
    hue="overall_survival_meta",
    ax=ax[2, 0]
)
# plot che correlation between age at diagnosis and overall_survival_months_meta
sns.scatterplot(
    data=clinical,
    x="age_at_diagnosis_meta",
    y="overall_survival_months_meta",
    hue="overall_survival_meta",
    ax=ax[2, 1]
)
# calculate the correlation between age at diagnosis and total mutations
corr_1 = clinical[["tumor_size_meta", "mutation_count_meta"]].corr(method="spearman")
# calculate the correlation between age at diagnosis and overall_survival_months_meta
corr_2 = clinical[["age_at_diagnosis_meta", "overall_survival_months_meta"]].corr(method="spearman")
# calculate p-value
p_vals = collections.defaultdict(list)
# Perform t-test
for target in ["age_at_diagnosis_meta", "mutation_count_meta", "tumor_size_meta"]:
    _, p_value = ttest_ind(
        clinical[clinical["overall_survival_meta"] == 'alive'][target].dropna(),
        clinical[clinical["overall_survival_meta"] == 'dead'][target].dropna(),
    )
    p_vals[target].append(p_value)
# set the title of the figure
ax[0,0].set(
    title=f"dist of age at diagnosis by survival | p-value: {p_vals['age_at_diagnosis_meta'][0]:.3f}"
    )
ax[0,1].set(
    title=f"dist of age at diagnosis by survival | p-value: {p_vals['mutation_count_meta'][0]:.3f}"
    )
ax[1,0].set(
    title=f"dist of total mutations by survival | p-value: {p_vals['tumor_size_meta'][0]:.3f}"
    )
ax[1,1].set(
    title=f"dist of age at diagnosis by survival | p-value: {p_vals['age_at_diagnosis_meta'][0]:.3f}"
    )
ax[2,0].set(
    title=f"corr of age at diagnosis and total mutations | corr: {corr_1.iloc[0,1]:.3f}"
)
ax[2,1].set(
    title=f"corr of age at diagnosis and overall survival | corr: {corr_2.iloc[0,1]:.3f}"
)
# show figure
plt.show()


Explore correlation across all clinical variables

In [ ]:
# select only numeric columns from clinical data
corr_data = clinical.select_dtypes(include=['number'])
# calculate the correlation matrix
clinical_corr = corr_data.corr(method='spearman')
# plot the correlation matrix with plotly
import plotly.express as px
# generate heatmap
fig = px.imshow(
    clinical_corr, 
    zmin=-1, 
    zmax=1
)
# show figure
fig.show()

## Survival Analysis

Survival analysis is a statistical method used to analyze the time until an event of interest occurs, such as death, failure, or censoring. The goal of survival analysis is to study the distribution of the time to the event and identify factors that influence the time to the event. It is commonly used in medical research to study time to death, time to failure in mechanical systems, and time to a certain outcome in clinical trials. Survival analysis takes into account the fact that not all individuals will experience the event of interest, and it uses methods such as the Kaplan-Meier estimate and Cox regression to account for censoring, which occurs when some individuals do not experience the event before the study ends.

In [ ]:
print(f'clinical data shape: {clinical.shape}')
survival_data = clinical[clinical['death_from_cancer_meta'] != "Died of Other Causes"].copy()
print(f'survival data shape: {survival_data.shape}')
# convert the survival type to boolean
survival_data['death_from_cancer_meta'] = survival_data['death_from_cancer_meta'].map({'Living': False, 'Died of Disease': True})
# remove rows with na values
survival_data.dropna(inplace=True, subset=['overall_survival_months_meta', 'death_from_cancer_meta'])
print(f'survival data shape: {survival_data.shape}')
# assign to data_x and data_y and convert to array-like
death_event = survival_data['death_from_cancer_meta'].to_list()
time_moth = survival_data['overall_survival_months_meta'].to_list()

Here we build a kaplan meier curve of survival with the package [scikit-survival](https://scikit-survival.readthedocs.io/en/stable/index.html)

In [ ]:
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter

# fit the kaplar-meier estimator
kmf = KaplanMeierFitter()
kmf.fit(time_moth, death_event)
# Plot the Kaplan-Meier curve
kmf.plot(figsize=(10,6), linewidth=2)
plt.title("Kaplan-Meier Survival Curve")
plt.xlabel("Time (days)")
plt.ylabel("Survival Probability")
plt.show()

Estimate survival by group.

In [ ]:
placeholder = survival_data.dropna(subset=['cellularity_meta']).copy()

event_col = 'death_from_cancer_meta'
time_col = 'overall_survival_months_meta'

group_col = 'cellularity_meta'
groups = placeholder[group_col].unique()

# Fit the Kaplan-Meier model for each group
kmf = KaplanMeierFitter()
fig, ax = plt.subplots(figsize=(10,6))
for group in groups:
    group_data = placeholder[placeholder[group_col] == group]
    kmf.fit(group_data[time_col], event_observed=group_data[event_col], label=group)
    kmf.plot(ax=ax, linewidth=2)

# Plot the Kaplan-Meier curve
plt.title("Kaplan-Meier Survival Curve by Group")
plt.xlabel("Time (days)")
plt.ylabel("Survival Probability")
plt.legend()
plt.show()

In [ ]:
placeholder = survival_data.dropna(subset=['cellularity_meta']).copy()

event_col = 'death_from_cancer_meta'
time_col = 'overall_survival_months_meta'

group_col = 'her2_status_meta'
groups = placeholder[group_col].unique()

# Fit the Kaplan-Meier model for each group
kmf = KaplanMeierFitter()
fig, ax = plt.subplots(figsize=(10,6))
for group in groups:
    group_data = placeholder[placeholder[group_col] == group]
    kmf.fit(group_data[time_col], event_observed=group_data[event_col], label=group)
    kmf.plot(ax=ax, linewidth=2)

# Plot the Kaplan-Meier curve
plt.title("Kaplan-Meier Survival Curve by Group")
plt.xlabel("Time (days)")
plt.ylabel("Survival Probability")
plt.legend()
plt.show()

## Cox's Proportion Hazard's Model

The Cox Proportional Hazards Model is a statistical model used to analyze the survival time of individuals or objects, taking into account the effects of one or more predictor variables. It is commonly used in medical research, engineering, and other fields. In the model, the hazard (the risk of an event occurring at a specific time) is assumed to be proportional over time for individuals with different values of the predictor variables. The model estimates the hazard ratios, which can be used to compare the risks of different groups of individuals or objects and to make predictions about future events.

Hazard ratios are a measure of the relative risk of an event (such as death or failure) occurring in one group compared to another. In the Cox Proportional Hazards Model, hazard ratios represent the exponential of the coefficients estimated for the predictor variables in the model. The hazard ratio for a predictor variable tells us how much the hazard (the risk of an event occurring) increases or decreases with a unit increase in that variable, while holding all other variables constant.

For example, if the hazard ratio for a predictor variable is 2, it means that, for individuals with a one-unit increase in that variable, the risk of the event occurring is two times higher than for individuals with a lower value of that variable. On the other hand, if the hazard ratio is 0.5, it means that the risk of the event is reduced by half for individuals with a one-unit increase in that variable. Hazard ratios are used to determine the effect of a predictor variable on the risk of an event, and they provide a useful tool for comparing the risk of different groups.

The terms Cox's Proportional Hazards Model and Cox Regression are used to describe the same statistical model, which is a semi-parametric regression model for analyzing time-to-event data. The key difference between the two terms is that Cox Regression refers to the linear regression version of the model, while Cox's Proportional Hazards Model refers to the model in its more general form, which includes non-linear as well as linear forms.


The log-likelihood is a measure of the goodness of fit of a statistical model. In the context of the Cox Proportional Hazards Model, the log-likelihood is a measure of how well the model fits the data in terms of the time-to-event information and the event information.

In general, the log-likelihood of a model is defined as the logarithm of the likelihood function, which is a function that describes the probability of observing the data given the parameters of the model. The likelihood function measures the compatibility of the observed data with the model, and the log-likelihood transforms this compatibility into a logarithmic scale, making it easier to work with and compare different models.

In the case of the Cox Proportional Hazards Model, the log-likelihood measures the compatibility of the model with the data in terms of the hazard function, which is the instantaneous risk of an event occurring at a given time. A high log-likelihood indicates that the model fits the data well, and a low log-likelihood indicates that the model does not fit the data well.

The log-likelihood is often used as a criterion for selecting the best model from a set of candidate models, or for estimating the parameters of a model in a maximum likelihood estimation procedure. When used in cross-validation, the log-likelihood can provide a measure of the performance of the model on new, unseen data.

Fit a Cox's Proportion Hazard model with dummy variables.

In [ ]:
from lifelines import CoxPHFitter
from sklearn.model_selection import StratifiedKFold
import numpy as np

# Specify the duration column and the event column
duration_col = "overall_survival_months_meta"
event_col = "overall_survival_meta"
# subset only numeric columns
#df = clinical.select_dtypes(include=['number']).copy()
df = clinical[[duration_col, event_col]].copy()
# recode the event column to numeric
df[event_col] = clinical[event_col].map({'alive': 0, 'dead': 1})
# remove rows with na values
df = df.dropna()
# Define the number of folds for cross-validation
k = 5
# Create the cross-validation object
kf = StratifiedKFold(n_splits=k, shuffle=True)
# Initialize a list to store the log-likelihoods from each fold
log_likelihoods = []
# Loop over the folds
for train_index, val_index in kf.split(df, df[event_col]):
    # Split the data into training and validation sets
    df_train, df_val = df.iloc[train_index], df.iloc[val_index]
    
    # Fit the Cox Proportional Hazards Model on the training data
    cph = CoxPHFitter()
    cph.fit(df_train, duration_col=duration_col, event_col=event_col)
    
    # Compute the log-likelihood of the model on the validation data
    #log_likelihood = cph.score(df_val)
    log_likelihood = cph.score(df_val, scoring_method='concordance_index')

    # Store the log-likelihood
    log_likelihoods.append(log_likelihood)

# Compute the mean and standard deviation of the log-likelihoods
mean_log_likelihood = np.mean(log_likelihoods)
# Print the results
print("Mean log-likelihood: {:.2f}".format(mean_log_likelihood))

# fit model with all data
cph = CoxPHFitter()
cph.fit(df, duration_col=duration_col, event_col=event_col)
# print the coefficients
cph.print_summary()

Fit the Cox's Proportion Hazard Model with all numeric variabls.

In [ ]:
from lifelines import CoxPHFitter
from sklearn.model_selection import StratifiedKFold
import numpy as np

# Specify the duration column and the event column
duration_col = "overall_survival_months_meta"
event_col = "overall_survival_meta"
# subset only numeric columns
df = clinical.select_dtypes(include=['number']).copy()
# recode the event column to numeric
df[event_col] = clinical[event_col].map({'alive': 0, 'dead': 1})
# remove rows with na values
df = df.dropna()
# Define the number of folds for cross-validation
k = 5
# Create the cross-validation object
kf = StratifiedKFold(n_splits=k, shuffle=True)
# Initialize a list to store the log-likelihoods from each fold
log_likelihoods = []
# Loop over the folds
for train_index, val_index in kf.split(df, df[event_col]):
    # Split the data into training and validation sets
    df_train, df_val = df.iloc[train_index], df.iloc[val_index]
    
    # Fit the Cox Proportional Hazards Model on the training data
    cph = CoxPHFitter()
    cph.fit(df_train, duration_col=duration_col, event_col=event_col)
    
    # Compute the log-likelihood of the model on the validation data
    log_likelihood = cph.score(df_val, scoring_method='concordance_index')
    
    # Store the log-likelihood
    log_likelihoods.append(log_likelihood)

# Compute the mean and standard deviation of the log-likelihoods
mean_log_likelihood = np.mean(log_likelihoods)
std_log_likelihood = np.std(log_likelihoods)

# Print the results
print("Mean log-likelihood: {:.2f}".format(mean_log_likelihood))
print("Standard deviation of log-likelihood: {:.2f}".format(std_log_likelihood))

# fit model with all data
cph = CoxPHFitter()
cph.fit(df, duration_col=duration_col, event_col=event_col)
# print the coefficients
cph.print_summary()

## High Dimension Data

StatsModel package link [here](https://www.statsmodels.org/stable/index.html).

In [ ]:
from sklearn.decomposition import PCA
# we do not need to scale the data because all 20000 features are measured on the same scale.
# from sklearn.preprocessing import StandardScaler

# transpose expression data
exp_t = expression.to_numpy().transpose()

# create a PCA model with 2 components
pca = PCA(n_components=4)

# fit the model to the data
pca.fit(exp_t)

# Access the components
X_pca = pca.components_.transpose()
# generate plot
fig, ax = plt.subplots(2,2, figsize=(20, 10))
# generate plot
sns.scatterplot(
    x=X_pca[ : , 0],
    y=X_pca[ : , 1 ],
    hue=clinical['overall_survival_meta'].fillna(value = "NA").to_list(),
    ax=ax[0, 0]
    )
# generate plot
sns.scatterplot(
    x=X_pca[ : , 0],
    y=X_pca[ : , 2 ],
    hue=clinical['overall_survival_meta'].fillna(value = "NA").to_list(),
    ax=ax[0, 1]
    )
# generate plot  
sns.scatterplot(
    x=X_pca[ : , 1],
    y=X_pca[ : , 2 ],
    hue=clinical['overall_survival_meta'].fillna(value = "NA").to_list(),
    ax=ax[1, 0]
    )
# generate plot
sns.scatterplot(
    x=X_pca[ : , 2],
    y=X_pca[ : , 0 ],
    hue=clinical['overall_survival_meta'].fillna(value = "NA").to_list(),
    ax=ax[1, 1]
    )

ax[0,0].set_title("PC1 vs PC2")
ax[0,1].set_title("PC1 vs PC2")
ax[1,0].set_title("PC2 vs PC3")
ax[1,1].set_title("PC3 vs PC1")


Create an heatmap visualizing all features and samples.

In [ ]:
# generate heatmap of expression data
sns.clustermap(exp_t)

### Can we identify a set of molecules with different distribution between event survival and not?

For this task, multivariate regression modelling will be used. Multivariate regression is a statistical method that models the relationship between multiple independent variables and a dependent variable. It allows us to understand how changes in one or more independent variables influence the dependent variable and can be used to make predictions. This method assumes that there is a linear relationship between the variables and can be represented mathematically as a linear equation with multiple coefficients. The coefficients are estimated using data and the method of least squares.

In [ ]:
import seaborn as sns

# plot the distribution of the cause of death
sns.boxplot(
    x='age_at_diagnosis_meta',
    y='death_from_cancer_meta', 
    data=clinical)

# count the number of patients in each category
clinical['death_from_cancer_meta'].value_counts()

Remove patients that died for other causes from the dataset.

In [ ]:
# remove rows in the category Died of Other Causes
expr = expression[clinical['death_from_cancer_meta'] != 'Died of Other Causes'].copy()
# copy clinical data
meta_df = clinical[clinical['death_from_cancer_meta'] != 'Died of Other Causes'].copy()
# create a new dataset of measurements and metadata
df = pd.concat([expr, meta_df], axis=1)
# print dimensions of the new dataset
print(df.shape)

The quantification of molecules is done with multivariate linear regression analysis. 

Multivariate linear regression analysis is a statistical method used to model the relationship between multiple independent variables (also known as predictors) and a single dependent variable (also known as response or outcome). The goal of this analysis is to find the best linear combination of independent variables that predicts the dependent variable, which is represented by an equation with a set of coefficients. The quality of the model is typically evaluated based on its ability to explain the variation in the dependent variable, and to make accurate predictions based on the independent variables.

The aim is to determine the effect of death_from_cancer_meta on the quantification of each molecule using a linear model, where age and death_from_cancer_meta are the independent variables and the quantification of the molecule is the dependent variable. In bioinformatics, this process is called differential expression analysis.

In [ ]:
# import packages
import statsmodels.api as sm
import statsmodels.formula.api as smf

from statsmodels.stats.multitest import multipletests

df = df.dropna(subset=[
    'death_from_cancer_meta', 
    'age_at_diagnosis_meta', 
    'brca1_expr'
    ])

X = pd.get_dummies(df[['death_from_cancer_meta', 'age_at_diagnosis_meta']])
y = df['brca1_expr']
# fit the model to ols
model = sm.OLS(
    y, 
    sm.add_constant(X)
    ).fit()

print(model.summary())

The False Discovery Rate (FDR) is a measure of the proportion of false positives among all the significant results (i.e., rejected null hypotheses) in a multiple hypothesis test. In other words, it is the expected proportion of false positive results among the significant results. FDR is used in statistical analysis to control the number of false positives and maintain the overall error rate at a desired level, usually set at a small value such as 0.05 or 0.01.

In [ ]:
pvals = model.pvalues[1:]
_, corrected_pvals, _, _ = multipletests(pvals, method='fdr_bh')

corrected_pvals

Automated the process to estimate fdr for all molecules in the dataframe.

In [ ]:
# import packages
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests

X = pd.get_dummies(df[['death_from_cancer_meta', 'age_at_diagnosis_meta']])

fdr_df = list()

for molecule in expression.columns:
    y = df[molecule]
    # fit the model to ols
    model = sm.OLS(
        y, 
        sm.add_constant(X)
        ).fit()
    # create sumamry table
    summary = model.summary2().tables[1]
    # retrieve p-values
    pvals = model.pvalues[1:]
    # add gene name
    summary['gene'] = molecule
    # correct p-values
    _, corrected_pvals, _, _ = multipletests(pvals, method='fdr_bh')
    # add corrected p-values to the dataframe
    summary = summary.join(pd.DataFrame(corrected_pvals, index=X.columns, columns=['pval_corrected']))
    # append to fdr_df
    fdr_df.append(summary)
summary = pd.concat(fdr_df)

In [ ]:
# sort summary by pval_corrected
summary = summary.sort_values('pval_corrected', ascending=True)
living_coeff = summary.loc['death_from_cancer_meta_Living']
fdr_genes = living_coeff[living_coeff['pval_corrected'] < 0.01]['gene']
print( f'total number of molecules with influence on survival {fdr_genes.shape}' )

Show graphic for the first 10 top genes. This is a visual inspection to understand how strong is the change.

In [ ]:
# Create a subplot figure with 5 rows and 5 columns
fig, ax = plt.subplots(2, 5, figsize=(20, 20))

# Iterate over the genes
for n in range(10):
    target = fdr_genes[n]
    # Generate boxplot of ready_to_print genes by death_from_cancer_meta
    row = n // 5
    col = n % 5
    sns.boxplot(
        x='death_from_cancer_meta',
        y=target,
        data=df,
        ax=ax[row, col]
    ).set_title(target)

In [ ]:
# create heatmpa of all genes
sns.clustermap(exp_t, cmap='viridis', z_score='none', figsize=(4,4))
# create heatmap of top 10 genes
sns.clustermap(df[fdr_genes[:10]], cmap='viridis', z_score='none', figsize=(4,4))


## Modelling Responder (survival) with machine learning

Here we use PyCaret to try several models and find the best performing. It is a prime on PyCaret, and you can find a nice tutorial [here](https://michael-fuchs-python.netlify.app/2022/01/01/automl-using-pycaret-classification/)

In [ ]:
# prepare the dataset for the training
model_df = df.drop(["overall_survival_meta", "overall_survival_months_meta"], axis=1)


In [ ]:
# Create an instance of the PyCaret setup function
from pycaret.classification import *
clf = setup(
    data = model_df, 
    target = 'death_from_cancer_meta', 
    session_id=123,
    normalize = True, 
    transformation = True, 
    ignore_low_variance = True,
    remove_multicollinearity = True, 
    multicollinearity_threshold = 0.95
    ) 

available_models = models()
available_models
 

In [ ]:
best_clf = compare_models(sort = 'AUC', include=['dt', 'rf', 'xgboost'])
print(best_clf)

In [ ]:
evaluation_best_clf = evaluate_model(best_clf)


In [ ]:
final_model = finalize_model(best_clf)
save_model(final_model, '../models/my_model')

Model interpretation with SHAP values

Please see documentation [here](https://shap.readthedocs.io/en/latest/)